In [ ]:
from pathlib import Path
import os

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input17.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input17_test.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input17_test2.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data 

# Part 1

In [ ]:
data = [[int(char) for char in row] for row in data]
num_rows = len(data)
num_cols = len(data[0])
print(num_rows, num_cols)

In [ ]:
# last_dirs is of the form e.g. (2, "N") -> last two steps were N but not the one before
# this reduces the size of the state space compared to using last_dirs of the form "SSE" (last three dirs)

def generate_next_dirs(last_dirs, next_dir):
    count, last_dir = last_dirs
    if last_dir != next_dir:
        next_dirs = (1, next_dir)
    else:
        next_dirs = (count + 1, next_dir)
    return next_dirs

def generate_moves(x, y, last_dirs):
    count, last_dir = last_dirs

    moves = []
    if x > 0 and last_dir != "S" and last_dirs != (3, "N"):
        next_dirs = generate_next_dirs(last_dirs, "N")
        moves.append((x - 1, y, next_dirs))
    if x < num_rows - 1 and last_dir != "N" and last_dirs != (3, "S"):
        next_dirs = generate_next_dirs(last_dirs, "S")
        moves.append((x + 1, y, next_dirs))
    if y > 0 and last_dir != "E" and last_dirs != (3, "W"):
        next_dirs = generate_next_dirs(last_dirs, "W")
        moves.append((x, y - 1, next_dirs))
    if y < num_cols - 1 and last_dir != "W" and last_dirs != (3, "E"):
        next_dirs = generate_next_dirs(last_dirs, "E")
        moves.append((x, y + 1, next_dirs))

    return moves

In [ ]:
max_num_iter = 1e9

start = (0, 0, (1, "X"))
end_loc = (num_rows - 1, num_cols - 1)

to_expand = [start]
state_to_cost_dict = {start: 0}
# state_to_best_path_dict = {start[:2]: []}
state_to_best_path_dict = {start: []}
min_end_state_cost = float("inf")
best_path = None

num_iter = 0
while num_iter < max_num_iter:
    
    if num_iter % 10000 == 0:
        print(num_iter)
        # print(len(to_expand))

    current_state = to_expand.pop()
    cost_so_far = state_to_cost_dict[current_state]
    moves = generate_moves(*current_state)
    for move in moves:
        x, y, (count, last_dir) = move

        move_cost = data[x][y]
        total_cost = cost_so_far + move_cost

        # if move not in state_to_cost_dict or total_cost < state_to_cost_dict[move]:
        # exclude inadmissible moves: no point going to e.g. 3E at x,y if its cost is at least as high as for 2E at x,y
        if (move not in state_to_cost_dict or total_cost < state_to_cost_dict[move]) and not any(state_to_cost_dict.get((x, y, (c, last_dir)), float("inf")) <= total_cost for c in range(1, count)):
            state_to_cost_dict[move] = total_cost
            # new_path = state_to_best_path_dict[current_state[:2]] + [move[:2]]
            # state_to_best_path_dict[move[:2]] = new_path
            new_path = state_to_best_path_dict[current_state] + [move]
            state_to_best_path_dict[move] = new_path
            to_expand.append(move)

            # expand lowest-cost state first
            # could also use PriorityQueue
            to_expand = sorted(to_expand, key=state_to_cost_dict.get, reverse=True)
            
            if move[:2] == end_loc and total_cost < min_end_state_cost:
                min_end_state_cost = total_cost
                best_path = new_path
                print(f"New best path, {min_end_state_cost = }, {best_path = }")

    if all(state_to_cost_dict[move] >= min_end_state_cost for move in to_expand):
        print(f"Solved: {num_iter = }, {min_end_state_cost = }")
        break

    num_iter += 1

# Part 2

In [ ]:
def generate_moves_new(x, y, last_dirs):
    count, last_dir = last_dirs

    moves = []

    # Need to keep moving in same direction if count less than 4
    # If new direction chosen it needs to be possible to move at least four times in the same direction

    if x > 0 and last_dir != "S" and last_dirs != (10, "N") and ((count >= 4 and x > 3) or last_dir == "N"):
        next_dirs = generate_next_dirs(last_dirs, "N")
        moves.append((x - 1, y, next_dirs))
    if x < num_rows - 1 and last_dir != "N" and last_dirs != (10, "S")  and ((count >= 4 and x < num_rows - 4) or last_dir == "S"):
        next_dirs = generate_next_dirs(last_dirs, "S")
        moves.append((x + 1, y, next_dirs))
    if y > 0 and last_dir != "E" and last_dirs != (10, "W")  and ((count >= 4 and y > 3) or last_dir == "W"):
        next_dirs = generate_next_dirs(last_dirs, "W")
        moves.append((x, y - 1, next_dirs))
    if y < num_cols - 1 and last_dir != "W" and last_dirs != (10, "E")  and ((count >= 4 and y < num_cols - 4) or last_dir == "E"):
        next_dirs = generate_next_dirs(last_dirs, "E")
        moves.append((x, y + 1, next_dirs))

    return moves

In [ ]:
max_num_iter = 1e9

start = (0, 0, (100, "X")) # count = 100 to allow initial moves to be found
end_loc = (num_rows - 1, num_cols - 1)

to_expand = [start]
state_to_cost_dict = {start: 0}
# state_to_best_path_dict = {start[:2]: []}
state_to_best_path_dict = {start: []}
min_end_state_cost = float("inf")
best_path = None

num_iter = 0
while num_iter < max_num_iter:
    # print(to_expand)
    
    if num_iter % 10_000 == 0:
        print(f"{num_iter = }")
        # print(len(to_expand))

    current_state = to_expand.pop()
    cost_so_far = state_to_cost_dict[current_state]
    moves = generate_moves_new(*current_state)
    # print(moves)
    for move in moves:
        x, y, (count, last_dir) = move

        move_cost = data[x][y]
        total_cost = cost_so_far + move_cost

        # Modified previous inadmissibility criterion:
        # exclude inadmissible moves: no point going to e.g. 8E at x,y if its cost is at least as high as for 6E at x,y (if count > 4)
        if (move not in state_to_cost_dict or total_cost < state_to_cost_dict[move]) and (count <= 4 or not any(state_to_cost_dict.get((x, y, (c, last_dir)), float("inf")) <= total_cost for c in range(4, count))):
            state_to_cost_dict[move] = total_cost
            # new_path = state_to_best_path_dict[current_state[:2]] + [move[:2]]
            # state_to_best_path_dict[move[:2]] = new_path
            new_path = state_to_best_path_dict[current_state] + [move]
            state_to_best_path_dict[move] = new_path
            to_expand.append(move)

            # expand lowest-cost state first
            # could also use PriorityQueue
            to_expand = sorted(to_expand, key=state_to_cost_dict.get, reverse=True)

            # expand by distance to goal (less efficient)
            # to_expand = sorted(to_expand, key=lambda x: x[0] + x[1])

            # Combine (too slow)
            # indices = range(len(to_expand))
            # indices_sorted_by_cost = [i for i, _ in sorted(enumerate(to_expand), key=lambda x: state_to_cost_dict[x[1]], reverse=True)]
            # index_cost_rank_score = [indices_sorted_by_cost.index(i) for i in indices]
            # indices_sorted_by_distance = [i for i, _ in sorted(enumerate(to_expand), key=lambda x: x[1][0] + x[1][1], reverse=True)]
            # index_distance_rank_score = [indices_sorted_by_distance.index(i) for i in indices]
            # index_rank_score_combined = [x + y for x, y in zip(index_cost_rank_score, index_distance_rank_score)]
            # to_expand = [to_expand[i] for i in sorted(indices, key=lambda i: index_rank_score_combined[i])]
            
            
            if move[:2] == end_loc and total_cost < min_end_state_cost:
                min_end_state_cost = total_cost
                best_path = new_path
                print(f"New best path, {min_end_state_cost = }, {best_path = }")

    if all(state_to_cost_dict[move] >= min_end_state_cost for move in to_expand):
        print(f"Solved: {num_iter = }, {min_end_state_cost = }")
        break

    num_iter += 1